##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [2]:
%pip install -q -U "google-genai>=1.0.0"
%pip install -q wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

In [3]:
import textwrap
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [4]:
from google.colab import userdata
from google import genai
API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=API_KEY)

In [5]:
# Define the model to be used in this notebook

MODEL_ID="gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}
model=f"models/{MODEL_ID}"

## Function Calling with the Gemini API

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Step 1: Define the Search Function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-2.0-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [6]:
from typing import List

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

# This is the actual function that would be called based on the model's suggestion
# Define the function with type hints and docstring
def wikipedia_search(search_queries: List[str]) -> List[str]:
  """Search wikipedia for each query and summarize relevant docs.
    Args:
        search_queries: The user query to search wikipedia.

    Returns:
        A list of relevant information from wikipedia based on the query.
  """
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []

  # Note: newer models have more limited quota for API calls
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-2.0-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content
        response = client.models.generate_content(model=model,
                                                  contents=textwrap.dedent(f"""\
                                                        Extract relevant information
                                                        about user's query: {query}
                                                        From this source:

                                                        {page}

                                                        Note: Do not summarize. Only Extract and return the relevant information
                                                        """),)
        urls = [url]

        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

      except:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results


In [7]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Retrieval-augmented generation', 'Vibe coding']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Large language model did not match with any page id, hence skipping.
Fetching page: "Retrieval-augmented generation"
Information Source: https://en.wikipedia.org/wiki/Retrieval-augmented_generation
Retrieval-augmented generation did not match with any page id, hence skipping.
Fetching page: "Vibe coding"
Information Source: https://en.wikipedia.org/wiki/Vibe_coding
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Retrieval-augmented_generation
     https://en.wikipedia.org/wiki/Vibe_coding


Here is what the search results look like:

In [8]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

> Based on the provided text, here's the relevant information about Large Language Models (LLMs):
> 
> *   Vibe coding is an AI-dependent programming technique where a person describes a problem in a few sentences as a prompt to a large language model (LLM) tuned for coding.
> *   The LLM generates software, shifting the programmer's role from manual coding to guiding, testing, and refining the AI-generated source code.
> *   The concept of vibe coding relies on LLMs, allowing programmers to generate working code by providing natural language descriptions rather than manually writing it.
> *   Karpathy's claim from 2023 that "the hottest new programming language is English", meaning that the capabilities of LLMs were such that humans would no longer need to learn specific programming languages to command computers.
> *   AI researcher Simon Willison said: "If an LLM wrote every line of your code, but you've reviewed, tested, and understood it all, that's not vibe coding in my book—that's using an LLM as a typing assistant."
> 
> Based on:
>   https://en.wikipedia.org/wiki/Vibe_coding

### Step 2: Automatic Function Calling (Python Only)

When using the Python SDK, you can provide Python functions directly as tools. The SDK automatically converts the Python function to declarations, handles the function call execution and response cycle for you. The Python SDK then automatically:

- Detects function call responses from the model.
- Call the corresponding Python function in your code.
- Sends the function response back to the model.
- Returns the model's final text response.

To use this, define your function with type hints and a docstring, and then pass the function itself (not a JSON declaration) as a tool:

Note: This approach only handles annotations of `AllowedType = (int | float | bool | str | list['AllowedType'] | dict[str, AllowedType])`

In [9]:
from google.genai import types

config = types.GenerateContentConfig(
    tools=[wikipedia_search]
) # Pass the function itself

## Generate supporting search queries

In order to have multiple supporting search queries to the user's original query, you will ask the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [10]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

In order to yield creative and a more random variety of questions, you will set the model's temperature parameter to a value higher. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

### Step 3: Enable Automatic Function Calling and Call the API

Now start a new chat with default value of `automatic_function_calling`. Using the default `disable=False`, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [12]:
chat = client.chats.create(model=model, config=config, history=[])

query = "Explain how deep-sea life survives."

response = chat.send_message(instructions.format(query=query))

Searching for "deep sea life adaptations"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea gigantism']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Deep sea did not match with any page id, hence skipping.
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Deep-sea fish did not match with any page id, hence skipping.
Fetching page: "Deep-sea gigantism"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_gigantism
Deep-sea gigantism did not match with any page id, hence skipping.
Searching for "deep sea food sources"
Related search terms: ['Deep-sea fish', 'Deep sea', 'Deep-sea community']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Searching for "hydrothermal vent ecosystems"
Related search terms: ['Hydrothermal vent', 'Scaly-foot gastropod', "Loki's Castle"]
Fetching page: "Hydrothermal vent"
Information Source: https://en.

In [13]:
to_markdown(response.text)

> Deep-sea life survives through a few key adaptations and energy sources. Since sunlight doesn't penetrate these depths, the primary sources of energy are marine snow (organic matter falling from upper layers), whale falls, and chemosynthesis at hydrothermal vents and cold seeps. Organisms have adapted to the extreme pressure, lack of sunlight, and scarcity of food through unique biological and physiological mechanisms. Hydrothermal vent ecosystems, like the one at Loki's Castle, support unique microbial communities that form the base of the food chain, utilizing chemicals like hydrogen sulfide, hydrogen, and methane.


Check for additional citations:

In [14]:
response.candidates[0].citation_metadata or 'No citations found'

'No citations found'

### Step 4: Understand Chat History

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [15]:
for content in chat.get_history():
    display(Markdown("###" + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print("Function call: {", part.function_call, "}")
        if part.function_response:
            print("Function response: {", part.function_response, "}")
    print("-" * 80)

###user:

You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: Explain how deep-sea life survives.


--------------------------------------------------------------------------------


###model:

Function call: { id=None args={'search_queries': ['deep sea life adaptations', 'deep sea food sources', 'hydrothermal vent ecosystems', 'deep sea pressure effects on life', 'deep sea bioluminescence', 'deep sea chemosynthesis', 'challenges of deep sea life', 'deep sea organisms', 'how do animals survive in the deep sea?', 'deep sea ecosystems']} name='wikipedia_search' }
--------------------------------------------------------------------------------


###user:

Function response: { id=None name='wikipedia_search' response={'result': ['The three main sources of energy and nutrients for deep sea communities are:\n\n*   Marine snow\n*   Whale falls\n*   Chemosynthesis at hydrothermal vents and cold seeps\n\nBased on:\n  https://en.wikipedia.org/wiki/Deep-sea_community', "*   **Location:** Mid-Atlantic Ridge, 73 degrees north, between Iceland and Svalbard. Depth of 2,352 metres (7,717 ft).\n\n*   **Discovery:** Discovered in 2008, were the most northerly black smoker vents known at the time.\n\n*   **Geology:** Located on the Gakkel Ridge. Sediment-influenced basalt-hosted hydrothermal field. Vent fluid recharge from oceanic sediments. 21 types of minerals identified. Oldest hydrothermal material estimated to be over 9,100 years old.\n\n*   **Mounds and Chimneys:** Two sulfide mounds (20–30 metres high, 150–200 metres across) joined into a single composite mound. Five active chimneys venting water up to 320 °C (608 °F). Chimneys are predominantly

###model:

Deep-sea life survives through a few key adaptations and energy sources. Since sunlight doesn't penetrate these depths, the primary sources of energy are marine snow (organic matter falling from upper layers), whale falls, and chemosynthesis at hydrothermal vents and cold seeps. Organisms have adapted to the extreme pressure, lack of sunlight, and scarcity of food through unique biological and physiological mechanisms. Hydrothermal vent ecosystems, like the one at Loki's Castle, support unique microbial communities that form the base of the food chain, utilizing chemicals like hydrogen sulfide, hydrogen, and methane.


--------------------------------------------------------------------------------


In the chat history you can see all 4 steps:

1. User: Asks the question about the total number of mittens.
2. Model: Determines that the `wikipedia_search` is helpful and sends a FunctionCall request to the user.
3. User: The Chat session automatically executes the function (due to `_automatic_function_calling` is enabled by default) and sends back a FunctionResponse with the calculated result.
4. Model: Uses the function's output to formulate the final answer and presents it as a text response.

## [Optional] Manually Execute the Function Call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [16]:
config  = {
        "tools": [wikipedia_search],
        "automatic_function_calling": {"disable": True}, # for manual execution
    }

chat    = client.chats.create(model=model, config=config, history=[])
response  = chat.send_message(instructions.format(query=query))

Initially the model returns a `FunctionCall`:

In [17]:
fc = response.candidates[0].content.parts[0].function_call
print(fc)
print(f"FunctionCall's name is {fc.name}")

id=None args={'search_queries': ['deep sea life adaptations', 'deep sea ecosystem', 'hydrothermal vents life', 'deep sea food sources', 'pressure adaptation deep sea', 'bioluminescence deep sea', 'deep sea creatures survival', 'challenges of deep sea life', 'how animals survive in deep sea', 'deep sea chemosynthesis']} name='wikipedia_search'
FunctionCall's name is wikipedia_search


Call the function with generated arguments to get the results.

In [18]:
if fc.name == "wikipedia_search":
    summaries = wikipedia_search(**fc.args)
    print(f"Function execution result: {summaries}")

Searching for "deep sea life adaptations"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea gigantism']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Deep sea did not match with any page id, hence skipping.
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Deep-sea fish did not match with any page id, hence skipping.
Fetching page: "Deep-sea gigantism"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_gigantism
Searching for "deep sea ecosystem"
Related search terms: ['Deep-sea community', 'Deep sea', 'Deep sea mining']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea mining"
Information Source: https://en.wikipedia.org/wiki/Deep_sea_mining
Deep sea mining did not match with any page id, hence skipping.
Searching for "hydrothermal vents life"
Related search terms: ['Hydrothermal vent', 'Lost City Hydrot

Now send the `summaries` to the model.

In [19]:
# Create a function response part
function_response_part = types.Part.from_function_response(
    name=fc.name,
    response={"result": summaries},
)

response = chat.send_message(function_response_part)

to_markdown(response.text)

> Deep-sea life survives in extreme conditions through a variety of adaptations. Here's a breakdown:
> 
> 1.  **Environmental Challenges:** The deep sea presents several challenges, including perpetual darkness, immense pressure (20 to 1,000 atm), and cold temperatures.
> 
> 2.  **Energy and Nutrient Sources:** The primary sources of energy are marine snow (particulate organic matter sinking from upper layers), occasional surface blooms, whale falls, and chemosynthesis at hydrothermal vents and cold seeps.
> 
> 3.  **Adaptations to Pressure:** Deep-sea organisms have evolved to survive extreme pressure.
> 
> 4.  **Deep-Sea Gigantism:** Some deep-sea animals exhibit gigantism, potentially due to colder temperatures, food scarcity, reduced predation, and increased dissolved oxygen. Examples include giant isopods and squids.
> 
> 5.  **Hydrothermal Vents:** At hydrothermal vents, bacteria use chemicals for energy through chemosynthesis, forming the base of the food web. These vents can have extreme temperature variations.
> 
> In summary, deep-sea life survives by adapting to the immense pressure, utilizing alternative food sources like chemosynthesis, and developing unique characteristics like gigantism.


## Re-ranking the search results

Helper function to embed the content:

In [52]:
from tqdm.auto import tqdm
from google.genai import types

tqdm.pandas()

from google.api_core import retry
import numpy as np

def make_embed_text_fn(model):

    @retry.Retry(timeout=300.0)
    def embed_fn(texts: list[str]) -> list[list[float]]:
        # Set the task_type to SEMANTIC_SIMILARITY and embed the batch of texts
        embeddings = client.models.embed_content(
            model=model,
            contents=texts,
            config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY"),
        ).embeddings
        return np.array([embedding.values for embedding in embeddings])

    return embed_fn


def create_embeddings(content: list[str]) -> np.ndarray:
    MODEL_ID = "embedding-001" # @param ["embedding-001", "text-embedding-004","gemini-embedding-exp-03-07"] {"allow-input":true, isTemplate: true}
    model = f"models/{MODEL_ID}"
    embed_fn = make_embed_text_fn(model)

    batch_size = 100  # at most 100 requests can be in one batch
    all_embeddings = []

    # Loop over the texts in chunks of batch_size
    for i in tqdm(range(0, len(content), batch_size)):
        batch = content[i:i + batch_size]
        embeddings = embed_fn(batch)
        all_embeddings.extend(embeddings)

    return np.array(all_embeddings).reshape(len(all_embeddings), -1)

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and so their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [53]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [62]:
search_res = create_embeddings(summaries)
embedded_query = create_embeddings([query])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity score:

In [63]:
sim_value = dot_product(search_res, embedded_query)
sim_value

array([[0.70439355],
       [0.71418499],
       [0.65326162],
       [0.74787755],
       [0.65874862],
       [0.7342544 ]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [64]:
to_markdown(summaries[np.argmax(sim_value)])

> The provided text focuses on sea monsters from folklore and alleged sightings, not on the survival of deep-sea creatures. Therefore, there is no relevant information in this source that addresses the user's query about "deep sea creatures survival."
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Sea_monster

### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-2.0-flash`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [57]:
res = client.models.generate_content(model=model,
                                contents=f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}""",)

to_markdown(res.text)

> Okay, so imagine the deep sea, right? It's this crushing darkness, colder than a penguin's backside, and food is scarcer than hen's teeth. So how does *anything* survive down there? Well, it's a masterclass in adaptation, that's how.
> 
> Firstly, think about the *pressure*. We're talking unimaginable weight pushing down on everything. Deep-sea creatures haven't just "gotten used to it," they've actually evolved to *need* it. Their bodies are built differently. Instead of rigid bones that would shatter, many have flexible skeletons made of cartilage, like sharks. Others, like certain jellyfish, are almost entirely water, so they can equalize the pressure inside and out. Their cells also have special adaptations, like [Placeholder Specific Protein Name], that prevents them from collapsing under the immense force.
> 
> Then there's the *lack of sunlight*. Photosynthesis? Forget about it. So, where does the energy come from? Well, much of the deep-sea ecosystem relies on what we call "marine snow" - this constant shower of dead organisms and waste falling from the sunlit surface layers. Imagine it like a slow, continuous buffet of [Placeholder Organic Compound]. Creatures have developed ingenious ways to filter this stuff out, with specialized [Placeholder Anatomical Feature] that act like tiny nets.
> 
> But the *really* interesting stuff happens around hydrothermal vents. These are like underwater volcanoes, spewing out superheated water and chemicals from the Earth's core. Instead of sunlight, these vents support entire ecosystems based on chemosynthesis. Certain bacteria, like [Placeholder Bacterial Species Name], can convert chemicals like hydrogen sulfide into energy. These bacteria then become the food source for other creatures, like giant tube worms that have symbiotic relationships where they house the bacteria in special organs called [Placeholder Organ Name]. It's like a completely separate, self-sustaining world powered by the Earth's heat.
> 
> Finally, you need to consider *reproduction* in such a sparse environment. Finding a mate is tough! So many deep-sea creatures have evolved bizarre adaptations. Some, like the anglerfish, use bioluminescence – light produced by chemical reactions – to lure prey and potential partners. The male anglerfish is often tiny compared to the female and permanently attaches itself to her, becoming essentially a parasitic sperm bank. It's a truly [Placeholder Adjective describing the mating practice] strategy.
> 
> In short, survival in the deep sea is all about adapting to the extremes. It's a world of pressure, darkness, and scarce resources, but life finds a way, often in the most extraordinary and unexpected forms. The deep sea is truly a testament to the power of evolution, and there are probably countless other adaptations we haven't even discovered yet. I suspect there are even [Placeholder Hypothetical Lifeform] down there somewhere!


Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [58]:
hypothetical_ans = create_embeddings([res.text])

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity scores to rank the search results

In [59]:
sim_value = dot_product(search_res, hypothetical_ans)
sim_value

array([[0.75814509],
       [0.81477369],
       [0.71075259],
       [0.71510846],
       [0.71162398],
       [0.72665937]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [60]:
to_markdown(summaries[np.argmax(sim_value)])

> Here's the extracted information about deep-sea ecosystems from the provided text:
> 
> *   **Definition:** A deep-sea community is any community of organisms associated by a shared habitat in the deep sea.
> *   **Exploration Status:** Deep-sea communities remain largely unexplored due to technological, logistical challenges, and expense.
> *   **Historical Beliefs:** It was long believed that little life existed in the deep sea due to the extreme conditions (high pressure, temperature extremes, absence of light). Research since the 19th century has demonstrated significant biodiversity.
> *   **Energy and Nutrient Sources:** The three main sources of energy and nutrients for deep sea communities are:
>     *   Marine snow
>     *   Whale falls
>     *   Chemosynthesis at hydrothermal vents and cold seeps
> *   **Environment:**
>     *   **Darkness:** The aphotic zone, a region of perpetual darkness, extends from the bottom of the photic zone (where sunlight penetrates) down to the seabed.
>     *   **Hyperbaricity:** Pressure increases by about one atmosphere every ten meters. Deep-sea organisms have evolved adaptations to survive extreme pressure. The range of pressure is from 20 to 1,000 atm.
>     *   **Temperature:** Below the thermocline, the water mass of the deep ocean is cold and far more homogeneous. At any given depth, the temperature is practically unvarying over long periods of time. Hydrothermal vents are the direct contrast with constant temperature.
>     *   **Salinity:** Salinity is constant throughout the depths of the deep sea with two notable exceptions: the Mediterranean Sea and Brine pools.
> *   **Zones (Depth-based):**
>     *   **Mesopelagic:** 200 to 1,000 meters; the "twilight zone" with limited light.
>     *   **Bathyal:** 1,000 to 4,000 meters; "the midnight zone" with no light.
>     *   **Abyssal:** 4,000 to 6,000 meters; perpetual darkness and immense pressure.
>     *   **Hadal:** Below 6,000 meters; the deepest trenches, like the Mariana Trench.
> *   **Energy Sources (Detailed):**
>     *   **Marine Snow:** Particulate organic matter sinking from the upper layers.
>     *   **Occasional Surface Blooms:** Sudden influx of nutrients from plankton/algal blooms.
>     *   **Whale Falls:** Carcasses providing a large amount of organic matter, supporting a succession of communities (scavengers, opportunistic colonizers, sulfophilic bacteria).
>     *   **Chemosynthesis:**
>         *   **Hydrothermal Vents:** Located at plate boundaries; bacteria use chemicals from vents for energy, forming the base of the food web.
>         *   **Cold Seeps:** Areas of fluid seepage (hydrogen sulfide, methane); often form brine pools.
> *   **Ecology:**
>     *   Deep sea food webs are complex, and aspects of the system are poorly understood. Predator-prey interactions are studied through direct observation, stomach content analysis, and biochemical analysis.
>     *   Deep sea mining has severe consequences for ocean ecosystems. The destruction of habitats, disturbance of sediment layers, and noise pollution threaten marine species.
> *   **Deep Sea Research:**
>     *   Less than 4% of the ocean floor has been explored.
>     *   Submarines like DSV Alvin are used for exploration.
>     *   Pressurized fish traps and capture devices (PERISCOP) are being developed to study deep-sea creatures alive at the surface.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Deep-sea_community

You have now created a search re-ranking engine using embeddings!

## Next steps

I hope you found this example helpful! Check out more examples in the [Gemini Guide](https://github.com/google-gemini/gemini-guide/) to learn more.